# BPO

In [1]:
from lltk.imports import *

In [2]:
PATH_RAW_TAR='/home/ryan/DH/data/stacks/raw/bpo.tar'

In [3]:
# RAW_XML_KEYS=[
# #  'ActionCode',
# #  'AlphaPubDate',
# #  'DateTimeStamp',
#  'RecordID',
#  'RecordTitle',
#  'EndPage',
#  'ISSN',
#  'Issue',
#  'LanguageCode',
#  'NumericPubDate',
#  'ObjectType',
#  'Pagination',
#  'Publisher',
#  'SourceType',
#  'StartPage',
# #  'Terms',
# #  'URLDocView',
# #  'Version',
#  'Volume'
# ]

RAW_XML_KEYS=[
#     'ActionCode',
#     'AlphaPubDate',
#     'DateTimeStamp',
    'NumericPubDate',
    'RecordID',
    'RecordTitle',
    'Publication_PublicationID',
    'Publication_Title',
    'Publication_Qualifier',
    'Publisher',
    'Volume'
    'Issue',
    'StartPage',
    'EndPage',
    'ObjectType',
    'ISSN',
    'LanguageCode',
    'SourceType',
    'num_words'
#     'URLDocView',
#     'Version',
]

In [4]:
C=load_corpus('BPO')

In [5]:
C.path_raw=PATH_RAW_TAR

In [6]:
def parse_record(xmld):
    from collections import OrderedDict
    if not 'Record' in xmld: return {}
    d=xmld['Record']
    odx={}
    for k,v in d.items():
        if type(v) in {str,int,float}:
            odx[k]=v
        elif type(v) in {dict,OrderedDict}:
            for kk,vv in v.items():
                if type(vv) in {str,int,float}:
                    odx[k+'_'+kk]=vv
#                 else:
#                     print('???',k,kk,vv)
#                     stop
#         else:
            #print('!?!?',v)
            #stop
#     if 'Terms' in d:
#         td=d['Terms']
#         odx['Terms']=[]
#         for k,v in td.items():
#             if type(v)==dict:
#                 x=v.get('FlexTermValue')
#                 odx['Terms'].append(x)
    return odx

In [7]:
def compile_meta(self):
    import xmltodict
    done=0
    with tarfile.open(self.path_raw) as tar:
        iterr=tqdm(tar,position=0)
        for i,member in enumerate(iterr):
            if member.name.endswith('.xml'):
                with tar.extractfile(member) as mf:
                    mxml=mf.read().decode('utf-8')
                metaxml=mxml.split('<FullText>')[0].strip()
                if not metaxml.endswith('</Record>'): metaxml+='</Record>'
                try:
                    xmld=xmltodict.parse(metaxml)
                except Exception:
                    print(metaxml)
                    stop
                metad=parse_record(xmld)
                metad['num_words']=0 if not '<FullText>' in mxml else len(mxml.split('<FullText>',1)[1].split('</FullText>')[0].strip().split())
                yield metad
                done+=1
                #iterr.set_description(f'Yielded {done} metadata records')

In [9]:
# writegen(C.path_metadata, lambda: compile_meta(C), header=RAW_XML_KEYS)

In [82]:
pmfull=C.path_metadata.replace('.csv','-full.csv')
df=pd.read_csv(pmfull)

In [83]:
df.Publication_Title.value_counts()

The Athenaeum                                                                                                 139952
Illustrated Sporting and Dramatic News                                                                        118781
The Tatler                                                                                                    111757
Saturday review of politics, literature, science and art                                                      102594
The Sketch                                                                                                     96063
                                                                                                               ...  
The Examiner: or, remarks upon papers and occurrences, Aug. 3, 1710-Aug. 10/17, 1710                               4
Weekly lampoon; or, Satyrical reflections on the last weeks publick news letters and observator, Oct. 1690         3
Miscellanies over claret, or, The friends to the tavern,  1697  

In [84]:
df['NumericPubDateYM']=df.NumericPubDate.apply(lambda x: str(x)[:6])
df['NumericPubDateY']=df.NumericPubDate.apply(lambda x: int(str(x)[:4]))
df

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209,190610,1906
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for profe...,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494,189712,1897
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762,188001,1880
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95,195003,1950
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844...",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35,189906,1899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381,190203,1902
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33,195809,1958
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65,191809,1918
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776,189709,1897


In [85]:
df.NumericPubDateY.describe()

count    2.869244e+06
mean     1.873467e+03
std      5.171958e+01
min      1.010000e+03
25%      1.846000e+03
50%      1.884000e+03
75%      1.908000e+03
max      1.997000e+03
Name: NumericPubDateY, dtype: float64

In [86]:
len(df[df.NumericPubDateY<1670])

840

In [87]:
df.ObjectType.value_counts()

Advertisement          502606
Review                 498268
General Information    224206
News                   145026
Poem                   135268
Correspondence         124578
Fiction                113028
Back Matter             70643
Front Matter            19899
Obituary                16275
Recipe                   3879
Undefined                 278
Name: ObjectType, dtype: int64

In [88]:
Counter(df.ObjectType)

Counter({'Advertisement': 502606,
         'General Information': 224206,
         nan: 1015290,
         'Poem': 135268,
         'News': 145026,
         'Review': 498268,
         'Back Matter': 70643,
         'Fiction': 113028,
         'Front Matter': 19899,
         'Obituary': 16275,
         'Correspondence': 124578,
         'Recipe': 3879,
         'Undefined': 278})

In [89]:
df[df.ObjectType.isna()]

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762,188001,1880
6,19140601,8601756,"LEITH HILL.-April 28, 29.",2916,The School music review : a monthly periodical...,London / New York,H. W. Gray Co.,NaN,15,15,NaN,2043-5991,ENG,Historical Periodicals,126,191406,1914
7,18670810,8585775,THE PUBLIC HEALTH.,1759,Examiner,London,John Hunt,NaN,509,509,NaN,2043-278X,ENG,Historical Periodicals,127,186708,1867
10,19000718,1638051235,TEA -PICKING IN CHINA,2032690,The Sketch,London,Illustrated London News,NaN,574,NaN,NaN,NaN,ENG,Historical Periodicals,484,190007,1900
12,18760729,9461774,THE WAR.,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,[n. pag.],124,NaN,0950-9852,ENG,Historical Periodicals,1756,187607,1876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869229,19131124,1693150466,TRAFFICKING WITH CATHAY,2032689,The Sphere,London,Illustrated London News,NaN,24,NaN,NaN,NaN,ENG,Historical Periodicals,778,191311,1913
2869231,19100814,1618709446,THE WAY OF THE WAR,2032697,The Tatler,NaN,NaN,NaN,214,NaN,NaN,NaN,ENG,Historical Periodicals,2514,191008,1910
2869232,18901004,1693124464,AMERICA CONTRA MUNDUM,2032691,The Graphic,London,Illustrated London News,NaN,370,NaN,NaN,NaN,ENG,Historical Periodicals,442,189010,1890
2869238,18860320,1693091903,BUYING OUR THE IRISH LANDLORDS,2032691,The Graphic,London,Illustrated London News,NaN,302,NaN,NaN,NaN,ENG,Historical Periodicals,393,188603,1886


In [91]:
df['id']=[
    f'{b}/{c}/{a}'.replace(' ','_') for a,b,c in tqdm(zip(df.Publication_PublicationID, df.NumericPubDateY, df.ObjectType.fillna('Unknown')),total=len(df),position=0)
]
df

100%|██████████| 2869244/2869244 [00:01<00:00, 1715145.78it/s]


,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY,id
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209,190610,1906,1906/Advertisement/2563
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for profe...,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494,189712,1897,1897/General_Information/2896
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762,188001,1880,1880/Unknown/13715
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95,195003,1950,1950/Advertisement/2032697
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844...",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35,189906,1899,1899/Poem/2567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381,190203,1902,1902/Advertisement/13715
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33,195809,1958,1958/Advertisement/2032697
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65,191809,1918,1918/Advertisement/2032689
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776,189709,1897,1897/Unknown/2032690


In [92]:
record2id = dict(zip(df.RecordID.apply(str), df.id))
# record2id

In [70]:
def compile_txt(self):
    with tarfile.open(self.path_raw) as tar:
        iterr=tqdm(tar,position=0)
        for i,member in enumerate(iterr):
            if member.name.endswith('.xml'):
                with tar.extractfile(member) as mf:
                    mxml=mf.read().decode('utf-8')
                metaxml=mxml.split('<FullText>')[0].strip()
                rid=metaxml.split('</RecordID>')[0].split('<RecordID>')[-1]
                idx=record2id.get(rid)
                if not idx: continue
                otxt='' if not '<FullText>' in mxml else mxml.split('<FullText>',1)[1].split('</FullText>')[0].strip()
                if otxt:   
                    ofnfn=os.path.join(self.path_txt, idx+'.txt')
                    odir=os.path.dirname(ofnfn)
                    ensure_dir_exists(odir)
                    with open(ofnfn,'a') as of:
                        of.write(f'####{rid}\n{otxt}\n\n\n\n')

In [71]:
# next(compile_txt(C))

In [93]:
# compile_txt(C)

In [94]:
# !find {C.path_txt}

In [95]:
df

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY,id
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209,190610,1906,1906/Advertisement/2563
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for profe...,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494,189712,1897,1897/General_Information/2896
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762,188001,1880,1880/Unknown/13715
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95,195003,1950,1950/Advertisement/2032697
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844...",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35,189906,1899,1899/Poem/2567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, scien...",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381,190203,1902,1902/Advertisement/13715
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33,195809,1958,1958/Advertisement/2032697
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65,191809,1918,1918/Advertisement/2032689
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776,189709,1897,1897/Unknown/2032690


In [96]:
df=df.sort_values(['NumericPubDate','RecordID'])
df

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY,id
1420972,10101,1618364638,Advertisements,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,239,NaN,Advertisement,NaN,ENG,Historical Periodicals,1155,10101,1010,1010/Advertisement/2032695
2254352,10101,1618365333,NEW MUSIC,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,760,NaN,Advertisement,NaN,ENG,Historical Periodicals,2124,10101,1010,1010/Advertisement/2032695
2729107,10101,1618367119,MAPPIN & WEBB Ltd.,2032690,The Sketch,London,Illustrated London News,NaN,11,NaN,Advertisement,NaN,ENG,Historical Periodicals,131,10101,1010,1010/Advertisement/2032690
834345,10101,1618367761,THE Old Etonians beat Edinburgh University by ...,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,422,NaN,NaN,NaN,ENG,Historical Periodicals,55,10101,1010,1010/Unknown/2032695
841938,10101,1618367834,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A ...",2032690,The Sketch,London,Illustrated London News,NaN,9,NaN,News,NaN,ENG,Historical Periodicals,64,10101,1010,1010/News/2032690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560275,19970619,1638106991,&quot;The Sketch&quot; Silver Points,2032690,The Sketch,London,Illustrated London News,NaN,321,NaN,News,NaN,ENG,Historical Periodicals,0,199706,1997,1997/News/2032690
796501,19970619,1638107569,BLANKETED!,2032690,The Sketch,London,Illustrated London News,NaN,335,NaN,News,NaN,ENG,Historical Periodicals,0,199706,1997,1997/News/2032690
2795678,19970619,1638108337,Art and Photography,2032690,The Sketch,London,Illustrated London News,NaN,329,NaN,News,NaN,ENG,Historical Periodicals,0,199706,1997,1997/News/2032690
379917,19970619,1638108874,THE WOMAN-ABOUT-TOWN,2032690,The Sketch,London,Illustrated London News,NaN,346,NaN,NaN,NaN,ENG,Historical Periodicals,1177,199706,1997,1997/Unknown/2032690


In [97]:
df_nodup=df.drop_duplicates('id')
df_nodup

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY,id
1420972,10101,1618364638,Advertisements,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,239,NaN,Advertisement,NaN,ENG,Historical Periodicals,1155,10101,1010,1010/Advertisement/2032695
2729107,10101,1618367119,MAPPIN & WEBB Ltd.,2032690,The Sketch,London,Illustrated London News,NaN,11,NaN,Advertisement,NaN,ENG,Historical Periodicals,131,10101,1010,1010/Advertisement/2032690
834345,10101,1618367761,THE Old Etonians beat Edinburgh University by ...,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,422,NaN,NaN,NaN,ENG,Historical Periodicals,55,10101,1010,1010/Unknown/2032695
841938,10101,1618367834,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A ...",2032690,The Sketch,London,Illustrated London News,NaN,9,NaN,News,NaN,ENG,Historical Periodicals,64,10101,1010,1010/News/2032690
2258888,10101,1618372563,Graphic,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,420,NaN,News,NaN,ENG,Historical Periodicals,0,10101,1010,1010/News/2032695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708576,19960226,1689079340,TURKEY,2032697,London Life,London,NaN,NaN,67,NaN,Advertisement,NaN,ENG,Historical Periodicals,233,199602,1996,1996/Advertisement/2032697
2764820,19960226,1689080163,BELGRAVIA DECLARES METER WAR,2032697,London Life,London,NaN,NaN,14,NaN,NaN,NaN,ENG,Historical Periodicals,894,199602,1996,1996/Unknown/2032697
2759505,19960226,1689080953,The Absorbing Fire,2032697,London Life,London,NaN,NaN,58,NaN,Review,NaN,ENG,Historical Periodicals,68,199602,1996,1996/Review/2032697
360076,19970619,1638085680,A Living Wedgwood,2032690,The Sketch,London,Illustrated London News,NaN,319,NaN,News,NaN,ENG,Historical Periodicals,0,199706,1997,1997/News/2032690


In [103]:
df_nodup['year']=df_nodup['NumericPubDateY']
df_nodup['author']=''
df_nodup['title']=df_nodup['RecordTitle']

fix_meta(df_nodup[[
    c for c in df_nodup.columns if c not in {'NumericPubDateY','RecordTitle','NumericPubDateYM'}
]]).to_csv(C.path_metadata,index=False)

In [102]:
pd.read_csv(C.path_metadata)

,id,year,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,NumericPubDateYM,NumericPubDateY
0,1010/Advertisement/2032695,1010,10101,1618364638,Advertisements,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,239,NaN,Advertisement,NaN,ENG,Historical Periodicals,1155,10101,1010
1,1010/Advertisement/2032690,1010,10101,1618367119,MAPPIN & WEBB Ltd.,2032690,The Sketch,London,Illustrated London News,NaN,11,NaN,Advertisement,NaN,ENG,Historical Periodicals,131,10101,1010
2,1010/Unknown/2032695,1010,10101,1618367761,THE Old Etonians beat Edinburgh University by ...,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,422,NaN,NaN,NaN,ENG,Historical Periodicals,55,10101,1010
3,1010/News/2032690,1010,10101,1618367834,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A ...",2032690,The Sketch,London,Illustrated London News,NaN,9,NaN,News,NaN,ENG,Historical Periodicals,64,10101,1010
4,1010/News/2032695,1010,10101,1618372563,Graphic,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,NaN,420,NaN,News,NaN,ENG,Historical Periodicals,0,10101,1010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50444,1996/Advertisement/2032697,1996,19960226,1689079340,TURKEY,2032697,London Life,London,NaN,NaN,67,NaN,Advertisement,NaN,ENG,Historical Periodicals,233,199602,1996
50445,1996/Unknown/2032697,1996,19960226,1689080163,BELGRAVIA DECLARES METER WAR,2032697,London Life,London,NaN,NaN,14,NaN,NaN,NaN,ENG,Historical Periodicals,894,199602,1996
50446,1996/Review/2032697,1996,19960226,1689080953,The Absorbing Fire,2032697,London Life,London,NaN,NaN,58,NaN,Review,NaN,ENG,Historical Periodicals,68,199602,1996
50447,1997/News/2032690,1997,19970619,1638085680,A Living Wedgwood,2032690,The Sketch,London,Illustrated London News,NaN,319,NaN,News,NaN,ENG,Historical Periodicals,0,199706,1997
